In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from spektral.layers import GINConv
from spektral.utils.sparse import sp_matrix_to_sp_tensor
import scipy.sparse
import numpy as np

# Function to compute adjacency and sparse tensor

def get_mediapipe_adj_tensor():
    adj = get_mediapipe_adjacency_matrix()  # your existing function
    adj_sp = scipy.sparse.csr_matrix(adj)
    return sp_matrix_to_sp_tensor(adj_sp)

# Build function
def build_gnn_face_emotion_model(
    N: int,
    F: int,
    n_out: int,
    l2_reg: float = 5e-4,
    learning_rate: float = 1e-3
) -> tf.keras.Model:
    """
    Builds and compiles the GNN face emotion classification model.

    Args:
        N: Number of nodes (e.g., 468).
        F: Feature dimensionality per node (e.g., 2 for x, y).
        n_out: Number of emotion classes.
        l2_reg: L2 regularization factor.
        learning_rate: Learning rate for the Adam optimizer.

    Returns:
        A compiled tf.keras.Model instance.
    """
    # Prepare adjacency as sparse tensor
    adj_tensor = get_mediapipe_adj_tensor()

    # Input for node features
    X_in = Input(shape=(N, F), name='node_features')

    # First GNN block
    x = GINConv(64, activation='relu', kernel_regularizer=l2(l2_reg))([X_in, adj_tensor])
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Second GNN block
    x = GINConv(32, activation='relu', kernel_regularizer=l2(l2_reg))([x, adj_tensor])
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Global pooling (flatten)
    x = Flatten()(x)

    # Dense layers
    x = Dense(256, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Output
    output = Dense(n_out, activation='softmax', name='emotion_output')(x)

    # Model
    model = Model(inputs=X_in, outputs=output)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Example usage:
# model = build_gnn_face_emotion_model(N=468, F=2, n_out=6)
# model.summary()


In [ ]:
# (Re)define your `build_model()` exactly as you did originally,
# including the `get_mediapipe_adjacency_matrix() + sp_matrix_to_sp_tensor` call.
model = build_gnn_face_emotion_model(N=468, F=2, n_out=6)  
model.load_weights("gnn_face_emotion2.weights.h5")

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from spektral.layers import GINConv
from spektral.utils.sparse import sp_matrix_to_sp_tensor
import scipy.sparse
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_TESSELATION
import time

def get_mediapipe_adjacency_matrix():
    """Create adjacency matrix from MediaPipe face mesh connections"""
    # MediaPipe face mesh has 468 landmarks (without iris when refine_landmarks=False)
    n_nodes = 468
    adj_matrix = np.zeros((n_nodes, n_nodes), dtype=np.float32)
    
    # Add edges based on MediaPipe face mesh connections
    for connection in FACEMESH_TESSELATION:
        i, j = connection[0], connection[1]
        if i < n_nodes and j < n_nodes:  # Ensure indices are valid for 468 landmarks
            adj_matrix[i, j] = 1.0
            adj_matrix[j, i] = 1.0  # Undirected graph
    
    # Add self-loops
    np.fill_diagonal(adj_matrix, 1.0)
    
    return adj_matrix

# --- Constants and Initializations ---
# Define the emotion labels in the same order as your training data
EMOTIONS = ["Angry", "Disgusted", "Happy", "Neutral", "Sad", "Surprised"]

# Emotion colors for display (BGR format for OpenCV)
EMOTION_COLORS = {
    "Angry": (0, 0, 255),      # Red
    "Disgusted": (0, 128, 0),  # Green
    "Happy": (0, 255, 255),    # Yellow
    "Neutral": (128, 128, 128), # Gray
    "Sad": (255, 0, 0),        # Blue
    "Surprised": (0, 165, 255) # Orange
}

# Load the trained model with improved method
try:
    print("Loading trained GNN model...")
    from model_utils import load_model_with_validation
    model, metadata, adj_tensor_loaded = load_model_with_validation("emotion_gnn_model.h5")
    
    if model is None:
        print("❌ Could not load model. Make sure you've trained the model first.")
        exit()
        
    print("✅ Model loaded successfully.")
    print(f"Model input shape: {model.input_shape}")
    print(f"Model output shape: {model.output_shape}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Make sure 'emotion_gnn_model.h5' exists and was trained with the GNN code.")
    exit()

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1, 
    refine_landmarks=False,  # Use 468 landmarks (no iris)
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.5
)

# Create adjacency matrix for the model
print("Creating adjacency matrix...")
adj_matrix = get_mediapipe_adjacency_matrix()
adj_sparse = scipy.sparse.csr_matrix(adj_matrix)
adj_tensor = sp_matrix_to_sp_tensor(adj_sparse)

print(f"Adjacency matrix shape: {adj_matrix.shape}")
print(f"Number of edges: {np.sum(adj_matrix > 0) // 2}")

# Prediction smoothing
prediction_history = []
history_size = 5  # Number of frames to average for smoothing

def normalize_mesh_points(mesh_points):
    """Normalize mesh points to match training data preprocessing"""
    mesh_points = np.array(mesh_points, dtype=np.float32)
    
    # Center the points
    center = np.mean(mesh_points, axis=0)
    mesh_points = mesh_points - center
    
    # Scale to unit variance
    scale = np.std(mesh_points)
    if scale > 0:
        mesh_points = mesh_points / scale
    
    return mesh_points

def draw_prediction_bars(frame, predictions, current_emotion):
    """Draw probability bars for all emotions"""
    bar_width = 200
    bar_height = 25
    start_x = 10
    start_y = 80
    
    for i, (emotion, prob) in enumerate(zip(EMOTIONS, predictions[0])):
        y_pos = start_y + i * (bar_height + 5)
        
        # Background bar
        cv2.rectangle(frame, (start_x, y_pos), (start_x + bar_width, y_pos + bar_height), 
                     (50, 50, 50), -1)
        
        # Probability bar
        fill_width = int(bar_width * prob)
        color = EMOTION_COLORS[emotion] if emotion == current_emotion else (100, 100, 100)
        cv2.rectangle(frame, (start_x, y_pos), (start_x + fill_width, y_pos + bar_height), 
                     color, -1)
        
        # Text label
        cv2.putText(frame, f"{emotion}: {prob:.1%}", (start_x + bar_width + 10, y_pos + 18), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

# --- Main Webcam Loop ---
print("Starting webcam...")
print("Press 'q' to quit, 's' to save current frame, 'r' to reset prediction history")

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

# Set camera properties for better performance
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)

frame_count = 0
fps_start_time = time.time()
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame_count += 1
    H, W, _ = frame.shape
    
    # Calculate FPS
    if frame_count % 30 == 0:
        fps_end_time = time.time()
        fps = 30 / (fps_end_time - fps_start_time)
        fps_start_time = fps_end_time

    # Flip frame horizontally for mirror effect
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame with MediaPipe
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        # Use the landmarks of the first detected face
        face_landmarks = results.multi_face_landmarks[0]
        
        # --- Extract and preprocess landmarks for the model ---
        # 1. Extract X, Y coordinates
        mesh_points = np.array(
            [[p.x * W, p.y * H] for p in face_landmarks.landmark],
            dtype=np.float32
        )
        
        print(f"Number of landmarks detected: {len(mesh_points)}")  # Should be 468
        
        # 2. Normalize the landmarks (same as training)
        normalized_points = normalize_mesh_points(mesh_points)
        
        # 3. Reshape for the model input (add batch dimension)
        model_input = np.expand_dims(normalized_points, axis=0)
        
        try:
            # --- Predict Emotion ---
            prediction = model.predict(model_input, verbose=0)
            
            # Add to prediction history for smoothing
            prediction_history.append(prediction[0])
            if len(prediction_history) > history_size:
                prediction_history.pop(0)
            
            # Average predictions for smoother results
            smoothed_prediction = np.mean(prediction_history, axis=0)
            emotion_index = np.argmax(smoothed_prediction)
            emotion_label = EMOTIONS[emotion_index]
            confidence = smoothed_prediction[emotion_index]
            
            # --- Display the result on the frame ---
            # Draw face bounding box
            x_min = int(np.min(mesh_points[:, 0])) - 20
            y_min = int(np.min(mesh_points[:, 1])) - 20
            x_max = int(np.max(mesh_points[:, 0])) + 20
            y_max = int(np.max(mesh_points[:, 1])) + 20
            
            # Draw bounding box with emotion color
            color = EMOTION_COLORS[emotion_label]
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 3)
            
            # Display the emotion label and confidence
            text = f"{emotion_label}: {confidence:.1%}"
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
            
            # Background for text
            cv2.rectangle(frame, (x_min, y_min - 40), 
                         (x_min + text_size[0] + 10, y_min), color, -1)
            
            # Text
            cv2.putText(frame, text, (x_min + 5, y_min - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Draw face mesh (optional - can be disabled for better performance)
            # mp_drawing.draw_landmarks(
            #     frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
            #     landmark_drawing_spec=None,
            #     connection_drawing_spec=mp_drawing.DrawingSpec(
            #         color=(0, 255, 0), thickness=1, circle_radius=1)
            # )
            
            # Draw prediction bars
            draw_prediction_bars(frame, [smoothed_prediction], emotion_label)
            
        except Exception as e:
            print(f"Prediction error: {e}")
            cv2.putText(frame, "Prediction Error", (20, 40), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    else:
        cv2.putText(frame, "No Face Detected", (20, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        # Clear prediction history when no face is detected
        prediction_history.clear()

    # Display FPS
    cv2.putText(frame, f"FPS: {fps:.1f}", (W - 120, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Display instructions
    cv2.putText(frame, "Press 'q' to quit, 'r' to reset", (10, H - 20), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Show the final frame
    cv2.imshow('Facial Emotion Recognition (GNN)', frame)

    # Handle key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        prediction_history.clear()
        print("Prediction history reset")
    elif key == ord('s'):
        filename = f"emotion_frame_{int(time.time())}.jpg"
        cv2.imwrite(filename, frame)
        print(f"Frame saved as {filename}")

print("Shutting down...")

# --- Cleanup ---
cap.release()
cv2.waitKey(1) 
cv2.destroyAllWindows()
cv2.waitKey(1)
face_mesh.close()

print("✅ Application closed successfully.")